In [1]:
#install.packages("singleCellHaystack")

In [1]:
getwd()
setwd('/data01/ch6845/MarcoPolo/')
getwd()

[1] "/data01/ch6845/MarcoPolo/experiments"

[1] "/data01/ch6845/MarcoPolo"

In [2]:
library(singleCellHaystack)

In [3]:
set.seed(1234)

In [4]:
library(Matrix)
library(Seurat)
library(mclust)
library(clustree)
#citation("mclust")

Package 'mclust' version 5.4.6
Type 'citation("mclust")' for citing this R package in publications.

Loading required package: ggraph

Loading required package: ggplot2



In [5]:
read_in_seurat <-function(save_name,method){
    exp_data=readMM(sprintf("datasets/extract/%s.data.counts.mm",save_name))
 
    exp_data_row=read.table(sprintf('datasets/extract/%s.data.row',save_name))$V1
    exp_data_col=read.table(sprintf('datasets/extract/%s.data.col',save_name))$V1    
    rownames(exp_data)=exp_data_row
    colnames(exp_data)=exp_data_col    

    metadatarow=read.table(sprintf('datasets/extract/%s.metadatarow.tsv',save_name),sep='\t')
    metadatacol=read.table(sprintf('datasets/extract/%s.metadatacol.tsv',save_name),sep='\t')  
    
    metadatacol['size_factor']=read.table(sprintf('datasets/extract/%s.size_factor.tsv',save_name),sep='\t')$V1

    seurat <- CreateSeuratObject(counts = exp_data, project = "scRNAseq", assay = "RNA",
                                         min.cells = 0, min.features = 0,
                                         meta.data = as.data.frame(metadatacol))    
    
    for(col in colnames(metadatarow)){
        #print(col)
        #print(head(metadatarow[rownames(seurat[['RNA']]@meta.features),][col]))
        seurat[['RNA']]@meta.features[col]=metadatarow[rownames(seurat[['RNA']]@meta.features),][col]
    }
    seurat
}

In [6]:
dataset_name_all=list(
'Kohinbulk_filtered',
'HumanLiver_filtered',
'Zhengmix8eq_filtered')

length(dataset_name_all)

[1] 3

In [7]:
for (prop in c('1e-2','5e-3','1e-3','5e-4')){
    for(i in c(1:10)){
        dataset_name<-sprintf('Simul_1_%s_%s_filtered',prop,i)
        dataset_name_all<-append(dataset_name_all,dataset_name)
    }
        
} 

In [6]:
tabula_name_all=list("TabulaAorta_filtered",
"TabulaBladder_filtered",
"TabulaBrainMyeloid_filtered",
"TabulaBrainNonMyeloid_filtered",
"TabulaDiaphragm_filtered",
"TabulaFat_filtered",
"TabulaHeart_filtered",
"TabulaKidney_filtered",
"TabulaLargeIntestine_filtered",
"TabulaLimbMuscle_filtered",
"TabulaLiver_filtered",
"TabulaLung_filtered",
"TabulaMammaryGland_filtered",
"TabulaMarrow_filtered",
"TabulaPancreas_filtered",
"TabulaSkin_filtered",
"TabulaSpleen_filtered",
"TabulaThymus_filtered",
"TabulaTongue_filtered",
"TabulaTrachea_filtered")

In [7]:
dataset_name_all=tabula_name_all

In [8]:
head(dataset_name_all)

[[1]]
[1] "TabulaAorta_filtered"

[[2]]
[1] "TabulaBladder_filtered"

[[3]]
[1] "TabulaBrainMyeloid_filtered"

[[4]]
[1] "TabulaBrainNonMyeloid_filtered"

[[5]]
[1] "TabulaDiaphragm_filtered"

[[6]]
[1] "TabulaFat_filtered"

In [9]:
for(dataset_name in dataset_name_all[1:length(dataset_name_all)]){
    print(dataset_name)
    
    seurat=read_in_seurat(dataset_name)
    
    seurat_hvg <- NormalizeData(seurat)

    all.genes <- rownames(seurat_hvg)
    seurat_hvg=ScaleData(seurat_hvg,features = all.genes)

    seurat_hvg <- FindVariableFeatures(seurat_hvg, selection.method = "vst", nfeatures = 2000)

    seurat_hvg_pca <- RunPCA(seurat_hvg, features = VariableFeatures(object = seurat_hvg))    
    seurat_hvg_pca_tsne <- RunTSNE(seurat_hvg_pca, dims= 1:10)    
    
    
    dat.expression<-seurat_hvg_pca[['RNA']]@counts
    dat.detection <- dat.expression > 1
    
    dat.tsne <-Embeddings(seurat_hvg_pca_tsne, reduction = "tsne")
    dat.pca <-Embeddings(seurat_hvg_pca_tsne, reduction = "tsne")
    
    res_2D <- haystack(dat.tsne, detection = dat.detection, method = "2D")
    res_highD <- haystack(dat.tsne, detection = dat.detection, method = "highD")

    
    write.table(res_2D$result[order(res_2D$result['log.p.adj']),],
                file=sprintf('datasets/extract/%s.hvg.ht.tsne.tsv',dataset_name),
                sep='\t',
                row.names=TRUE, 
                col.names=TRUE)   
    
    write.table(res_highD$result[order(res_highD$result['log.p.adj']),],
                file=sprintf('datasets/extract/%s.hvg.ht.pca.tsv',dataset_name),
                sep='\t',
                row.names=TRUE, 
                col.names=TRUE)       
    
}

[1] "TabulaAorta_filtered"


Centering and scaling data matrix

PC_ 1 
Positive:  Fabp4, Gpihbp1, Cd300lg, 1810011O10Rik, Tmem88, Eltd1, Cd36, Pecam1, S1pr1, Rsad2 
	   Ednrb, Cd93, Btnl9, 8430408G22Rik, Tinagl1, Mmrn2, Scarb1, Ushbp1, Icam2, Mgll 
	   Apold1, Robo4, Aqp7, Snrk, Tcf15, Cyp4b1, Cldn5, Ehd3, Sh2d3c, Epb4.1 
Negative:  Serping1, Col1a2, Col6a2, Dcn, Timp2, Col3a1, Col6a1, Serpinf1, Col1a1, Pcolce 
	   Gsn, Dpt, C1ra, Loxl1, Mfap5, Htra3, Islr, Aebp1, Ogn, Sod3 
	   Rarres2, Rcn3, Olfml3, Lrp1, C1s, Olfml2b, Ccdc80, Fbln1, Ddr2, Clec3b 
PC_ 2 
Positive:  Sparc, Fabp4, Tm4sf1, Gpihbp1, Tinagl1, Pecam1, Tsc22d1, Mgll, 1810011O10Rik, Tmem88 
	   Eltd1, Mmrn2, Cav1, S1pr1, Slc43a3, Sdpr, Cd93, Sparcl1, Jam2, Snrk 
	   Icam2, Tspan7, Cd300lg, Ednrb, Tgm2, Ecscr, Ehd2, Ushbp1, 8430408G22Rik, Btnl9 
Negative:  Cd74, Cd52, H2-Eb1, Lsp1, H2-Aa, Laptm5, Ctss, Sfpi1, Lcp1, H2-Ab1 
	   Coro1a, H2-DMa, Ptprc, Plbd1, Selplg, Irf5, Ifi30, Tyrobp, Pld4, Cd53 
	   Ly86, H2-DMb1, Napsa, Slamf7, Cd68, Itgb2, Cytip, Alox

[1] "TabulaBladder_filtered"


Centering and scaling data matrix

PC_ 1 
Positive:  Aqp3, Ly6d, Gsta4, Krt18, Krt15, Krt19, Krt8, Gsto1, Cldn7, Sfn 
	   Cldn4, Serinc2, Foxq1, Spint1, Mgst3, Klf5, Sprr1a, 1110032A04Rik, Gstm1, Ctse 
	   Tacstd2, Upk1b, Tmprss2, Ivl, Lgals3, Lmo1, Cbr2, Ldhb, Gata3, Lsr 
Negative:  Serping1, Sparc, Dcn, Htra1, Vim, Rcn3, Mxra8, Col1a1, Bgn, Pcolce 
	   Dkk3, Fstl1, Nbl1, C1ra, Ecm1, Prelp, Mmp2, Fbln5, Ppic, Fbln1 
	   Serpinh1, Fcgrt, Wnt2, Lgals9, Ctsk, Id3, Cebpd, Emp3, Serpinf1, Cd81 
PC_ 2 
Positive:  Clec3b, Scara5, Mfap5, Cd34, Sparcl1, Npy1r, Abi3bp, Pi16, Lum, Entpd2 
	   Osr1, Lsamp, Sfrp1, Hspa12b, Prss23, Inmt, Lbp, Gsn, Gpc3, Itm2a 
	   Arap1, Igfbp4, Col14a1, Nsg1, Dmkn, Tnxb, Fndc5, Gm12824, Serpina3g, Cfh 
Negative:  Tnc, Car3, Palld, Itga11, Pde4b, Rbp4, Hhip, Bmp5, Aldh1a2, Cpz 
	   Dkk2, 3632451O06Rik, Cxcl14, Spon1, Tspan13, Fam105a, Gas6, Tgfbi, Gpx3, Prss35 
	   Plk2, Pdgfrl, 2010002N04Rik, Myl9, Plau, Notum, Chst2, Moxd1, Col12a1, Col6a3 
PC_ 3 
Positive:  Ftl1

[1] "TabulaBrainMyeloid_filtered"


...

Centering and scaling data matrix

PC_ 1 
Positive:  Shisa5, H2-K1, Ltb, B2m, Ccnd2, Gimap3, Ms4a4b, H2-D1, Ccr7, Cnn2 
	   Itgb7, Vim, Bcl2, Il7r, Tgtp2, S1pr1, Tapbp, Ctss, Ctsw, Id2 
	   Tspan32, Stat1, Gpr18, Itm2a, Cd5, Sepp1, Rhof, S1pr4, Il27ra, Ifit1 
Negative:  Top2a, Mki67, Ccna2, Birc5, Rrm2, 2810417H13Rik, Cdk1, Kif11, Spc24, Cdca3 
	   Hist1h2ab, Spc25, Nusap1, Hmgb2, Ube2c, Ccnb2, Fbxo5, Plk1, Aurkb, Pbk 
	   Uhrf1, Tpx2, Ccnf, Ncapg, Cdca8, Tacc3, Hist1h2ae, Mxd3, Tyms, Ccnb1 
PC_ 2 
Positive:  H2-K1, Shisa5, B2m, Ccnd2, H2-D1, Ms4a4b, Cnn2, Ccr7, Vim, Gimap3 
	   Itgb7, Tuba4a, Tapbp, Bcl2, Ctsw, Stat1, Capg, Ctss, Lcp1, AW112010 
	   Hspa5, Il7r, Tgtp2, Itm2a, Ran, Id2, Tuba1b, Nkg7, Cd82, Eif5a 
Negative:  Rag1, Dntt, Cd8b1, Glcci1, Cd8a, Aqp11, Ldhb, Rmnd5a, Rag2, Ccr9 
	   Cd4, Slc16a5, Arl5c, Dhrs3, Fbxl12, Rorc, Xrcc6, Ly6d, Ets2, Lipa 
	   Btg2, Atl2, Trp53inp1, Gtf2h4, Sla, Clk3, Bcl2l1, Edem1, Ssbp2, Xrra1 
PC_ 3 
Positive:  Itm2a, Cd5, Tox, Cd6, Tubb5, Tuba4a,

[1] "TabulaTongue_filtered"


Centering and scaling data matrix

PC_ 1 
Positive:  Crip1, Hmgb1, Krt15, Stmn1, Wfdc2, Apoe, Hmgn2, Ftl1, 5730469M10Rik, Ptma 
	   Col17a1, H2afv, Cyb5r3, Calm2, Oat, Itgb4, Igfbp2, Ifitm3, S100a6, Ccnd2 
	   Anxa5, Igfbp7, Ckap2l, Serpinb6a, H2afz, Moxd1, Hmgb2, Bcam, Gclm, Cav1 
Negative:  Sbsn, Lypd3, Fam25c, Perp, Dsg1b, Cyp4f39, Krtdap, Acer1, Tgm3, Cst6 
	   Elovl4, Ly6d, Mt4, Ly6g6e, Dsg1a, Ephx3, Rhov, Sprr1a, Sult2b1, Psapl1 
	   2310033E01Rik, Endou, Rhbg, 2310002J15Rik, Ly6g6c, Cldn4, Rab25, Tgm1, Clic3, Krt16 
PC_ 2 
Positive:  Hn1, Birc5, Cdk1, Hmgb2, Ube2c, Tacc3, Cdc20, Ccna2, Ccnb1, Cdca8 
	   Cdca3, Racgap1, Tuba1c, Top2a, Aurkb, Kif22, Smc2, Spc24, Ccnb2, Cks1b 
	   Kpna2, Cks2, Nusap1, Pbk, Prc1, Arl6ip1, Plk1, Cenpa, Tubb4b, Mad2l1 
Negative:  Dst, Col17a1, Plxna2, Lamb3, Itgb4, S100a6, Sema3c, Lama3, Cxcr7, Slc6a6 
	   Igfbp3, Tgfbi, Krt14, Ccnd1, Lars2, Cux1, Slc39a6, Bcam, Gm12824, Tns4 
	   Jub, Col18a1, Slc12a4, Fam117b, Slc39a10, Slc7a1, Leprel1, Nrp2, Npnt, 

[1] "TabulaTrachea_filtered"


Centering and scaling data matrix

PC_ 1 
Positive:  Gsn, Dcn, Col1a2, Fstl1, Sparc, Col6a2, Col6a1, Mmp2, Serpinf1, Col3a1 
	   Pcolce, Islr, Col1a1, Mfap5, Nbl1, Gfpt2, Htra3, Fbn1, Sod3, Igfbp4 
	   Lum, Col5a2, Clec3b, Il11ra1, Ccdc80, Abi3bp, Igfbp6, Aebp1, Rarres2, Fbln1 
Negative:  Laptm5, Ucp2, Lcp1, Srgn, Cd53, Ptprc, Coro1a, Ctss, Cd74, Fcer1g 
	   Tyrobp, H2-Ab1, Sfpi1, H2-Aa, H2-Eb1, Cd83, Ly86, Cd52, H2-DMa, Pld4 
	   Cd68, Itgb2, Plek, Selplg, Hcls1, Cd14, H2-DMb1, Cytip, Bcl2a1b, Alox5ap 
PC_ 2 
Positive:  Lsp1, Axl, Cd44, Tgfbi, Gm2a, Ctss, Gsn, 5430435G22Rik, Mfap5, Clec3b 
	   Ly86, Cd83, H2-Eb1, Serpinf1, Ecm1, H2-Aa, Sfpi1, Tyrobp, Pld4, H2-Ab1 
	   Fcer1g, Cd68, Col3a1, Cd74, H2-DMa, Ctsc, Laptm5, Plek, Cybb, Igfbp4 
Negative:  Esam, Flt1, Eltd1, Pecam1, Emcn, Gpr116, Apold1, Tie1, Egfl7, Kdr 
	   Ptprb, Mmrn2, Rasip1, S1pr1, Cdh5, Gpihbp1, Podxl, Cd300lg, Robo4, Tinagl1 
	   Slc9a3r2, Gpr56, Scarb1, Epas1, Sox7, Fabp4, Hspb1, Bcam, Tgtp1, Cd200 
PC_ 3 
Positive:  

In [11]:
res_2D

$results
           D_KL  log.p.vals     log.p.adj T.counts
2808 2.68398920  -5.1251452  -1.949053906      999
1217 0.00000000   0.0000000   0.000000000     1000
3969 0.00000000   0.0000000   0.000000000     1000
2244 0.00000000   0.0000000   0.000000000     1000
1469 0.00000000   0.0000000   0.000000000     1000
587  0.00000000   0.0000000   0.000000000     1000
703  0.00000000   0.0000000   0.000000000     1000
1101 0.00000000   0.0000000   0.000000000     1000
1866 2.68398920  -5.1251452  -1.949053906      999
1090 0.00000000   0.0000000   0.000000000     1000
3273 0.00000000   0.0000000   0.000000000     1000
1118 0.00000000   0.0000000   0.000000000     1000
2311 0.00000000   0.0000000   0.000000000     1000
3442 0.00000000   0.0000000   0.000000000     1000
2943 2.68398920  -5.1251452  -1.949053906      999
4256 0.00000000   0.0000000   0.000000000     1000
1268 0.00000000   0.0000000   0.000000000     1000
2415 2.68398920  -5.1251452  -1.949053906      999
448  2.68398920  -5.12

In [14]:
haystack(dat.tsne, detection = dat.detection, method = "2D")

### calling haystack_2D()...



ERROR: Error in haystack_2D(x[, dim1], x[, dim2], detection = detection, use.advanced.sampling = use.advanced.sampling, : The number of columns in 'detection' must be the same as the length of 'x'


In [22]:
dataset_name

[1] "Kohinbulk_filtered"

In [21]:
res_2D$result[order(res_2D$result['log.p.adj']),]

,D_KL,log.p.vals,log.p.adj,T.counts
,<dbl>,<dbl>,<dbl>,<int>
ENSG00000069482,0.8377246,-18.49607,-14.32893,267
ENSG00000006128,0.8421037,-18.14164,-13.97450,177
ENSG00000121966,0.8438000,-17.26487,-13.09773,217
ENSG00000168542,1.0390340,-17.16285,-12.99571,312
ENSG00000136110,0.6878173,-16.81286,-12.64572,268
ENSG00000146374,0.6926467,-16.79664,-12.62950,252
ENSG00000204335,0.6818064,-16.76082,-12.59368,256
ENSG00000205358,0.7682201,-16.68247,-12.51533,163
ENSG00000106038,0.7001881,-16.49548,-12.32834,172


In [20]:
res_highD$result[order(res_highD$result['log.p.adj']),]

,D_KL,log.p.vals,log.p.adj,T.counts
,<dbl>,<dbl>,<dbl>,<int>
ENSG00000121966,1.0071110,-27.63031,-23.46317,217
ENSG00000069482,1.0587212,-26.67426,-22.50712,267
ENSG00000254934,1.1477071,-26.23802,-22.07088,303
ENSG00000147869,0.9441783,-25.37861,-21.21147,221
ENSG00000204335,0.9904978,-25.25883,-21.09169,256
ENSG00000146374,0.9783368,-25.14600,-20.97886,252
ENSG00000128918,0.9086595,-24.05097,-19.88383,216
ENSG00000168542,1.1092045,-23.90070,-19.73356,312
ENSG00000136110,0.9771831,-23.84964,-19.68250,268


In [ ]:
|